### Make separate snakefile to process viruses
- After atlas already done: contigs present. 
- Run genomad: get plasmids and viruses

We have for viruses and plasmids: 
- predicted genes (can make a phylo tree from the TerL if needed..)
- the contigs and proteins

Concatenate all plasmids and all viruses
- drep them, probably at 95% for viruses, and 99% for plasmids? Can do 99 for both. 
- Figure out what plasmid ones are also in MAGs. 

In [ ]:
# genomad done!!
# I think we want to keep plasmids and viruses with a score > 0.9
# rename contigs, then concatenate


# cat all the score things too
csvtk concat -t */*_contigs_summary/*_contigs_plasmid_summary.tsv \
> /group/ctbrowngrp2/amhorst/2025-pigparadigm/results/plasmids/250417_plasmid_summary.tsv

csvtk concat -t */*_contigs_summary/*_contigs_virus_summary.tsv \
> /group/ctbrowngrp2/amhorst/2025-pigparadigm/results/viral/250417_viral_summary.tsv



In [ ]:
genomad end-to-end \
--cleanup --splits \
8 GCF_009025895.1.fna.gz \
genomad_output genomad_db

In [ ]:
#GENOMAD
# srun it, needs quite some mem
srun --account=ctbrowngrp -p med2 -J rename -t 1:00:00 -c 30 --mem=40gb --pty bash

# end to end for everything, need to annotate for classification
# Use for taxonomy of DNA phage
# softlink db
ln -s /group/jbemersogrp/databases/genomad/genomad_db .

# own genomad db
/home/amhorst/databases/genomad/genomad_db

# running 
srun --account=ctbrowngrp -p high2 -J gmd -t 12:00:00 -c 120 --mem=100gb --pty bash
mamba activate branchwater-skipmer

srun --account=ctbrowngrp -p high2 -J gmd -t 1:00:00 -c 25 --mem=20gb --pty bash
mamba activate branchwater-skipmer

snakemake \
-s genomad.smk \
--rerun-triggers mtime --rerun-incomplete \
--resources mem_mb=20000 -c 25 --use-conda \
-k -n


In [ ]:
# without contigs:
SRR11183349

In [ ]:
# running atlas in a screen, still wil submit jobs to cluster
srun --account=ctbrowngrp -p med2 -J vs2 -t 48:00:00 -c 36 --mem=50gb --pty bash
mamba activate branchwater-skipmer

snakemake \
-s virsorter2.smk \
--rerun-triggers mtime --rerun-incomplete \
--resources mem_mb=50000 -c 36 --use-conda \
-k -n


In [ ]:
# another snakefile for renaming and using the quality tresh. 

# rename found vOTUs
rule bbmap_rename_viral:
    input: 
        check = "../results/check/{sample}_vs2_done.check"
    output: 
        contigs = "../results/virsorter2/contigs/{sample}_rename.fa",
        check = "../results/check/{sample}_rename.check"
    log:
        "../results/logs/virsorter2/{sample}_rename.log"
    conda: 
        "bbmap"
    shell:
        """
    if [[ ! -e ../results/virsorter2/{wildcards.sample}/final-viral-combined.fa ]] 
    then
        touch {output.contigs} 
    else
        rename.sh in=../results/virsorter2/{wildcards.sample}/final-viral-combined.fa \
        out={output.contigs} prefix=AtH2023_{wildcards.sample}_viral && touch {output.check}
    fi
        """


